In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

PREPARAMOS LOS DATOS

In [ ]:
size =(256, 256)
#modelo segmentacion
path_modelo_segmentacion = "./modeloPropio.h5"
modelo_propio = keras.models.load_model("modeloPropio.h5")


#

In [ ]:
def prepare_image_for_segmentation(path,size=size,contrast=False,noise=False):
    im = cv2.imread(path,0)
    im = cv2.resize(im,size)

    
    #delete noise
    if noise:
        im = cv2.GaussianBlur(im,(5,5),0)

    if contrast:
        #increase contrast
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        im = clahe.apply(im)
    
    
    
    im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    im = im.reshape((1,)+im.shape)
    
    return im

In [ ]:
def obtain_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(i, axis=-1)
    mask = np.expand_dims(mask,axis=-1)
    #if np.count_nonzero(mask == 0) < np.count_nonzero(mask == 1):
    #    mask = np.invert(mask)
    #mask = mask.astype(np.uint8)
    mask = mask.reshape(size)
    mask = mask*255
    mask = mask.astype(np.uint8)

    return mask

In [ ]:
def watershed(predicted_mask,img):
    #convert to gray the image
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((3,3),np.uint8)  
    #opening
    predicted_mask = cv2.morphologyEx(predicted_mask, cv2.MORPH_OPEN, kernel, iterations = 2)
    #sure background area  
    sure_bg = cv2.dilate(predicted_mask,kernel,iterations=10)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(predicted_mask,cv2.DIST_L2,0)
    ret, sure_fg = cv2.threshold(dist_transform,0.15*dist_transform.max(),255,0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    
    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,0,0]

    #ALL THE PIXELS THAT ARE NOT 0 ARE 255
    markers[markers != 1] = 255
    markers[markers == 1] = 0
    #put all the the pixels from the edges of the image to 255
    #Para que al hacer el watershed se quede pegado a los bordes y al separar solo quede la zona de interes
    #En el otro jupyter lo ponemos a 0 para evitar este efecto
    markers[0,:] = 0
    markers[-1,:] = 0
    markers[:,0] = 0
    markers[:,-1] = 0

    markers = markers.astype(np.uint8)

    return img,markers

In [ ]:
#directorio de las imagenes
path = "path_of_images"

#directorio de las mascaras finales
save_path = "path_of_final_masks"
            

In [ ]:
import matplotlib.image as imag_save

In [ ]:
#remove all directories from save_path
for i in os.listdir(save_path):
    #remove all files from the directory
    for j in os.listdir(save_path+"/"+i):
        os.remove(save_path+"/"+i+"/"+j)
    os.rmdir(save_path+"/"+i)

for i in os.listdir(path):
    
    #check if is a folder skip if not
    if not os.path.isdir(path+"/"+i):
        continue
    
    print("Analizando carpeta: "+str(i))
    os.mkdir(save_path+"/"+i)
    #iterate over all images in the subfolder
    for j in os.listdir(path+"/"+i):
        img = prepare_image_for_segmentation(path+"/"+i+"/"+j,size=size,noise=True)
        pred = modelo_propio.predict(img)
        mask = obtain_mask(pred)
        #invert mask if there are more 0 than 1
        if np.count_nonzero(mask == 0) < np.count_nonzero(mask == 1):
            mask = np.invert(mask)
        
        #watershed
        mask = watershed(mask,img[0])[1]
        imag_save.imsave(save_path+"/"+i+"/"+j, mask,cmap="gray")


In [ ]:
def separar_regiones(img,segmented=None,path="./prueba/",j=0):
    
    if segmented is None:
        segmented = img.copy()
    #put all the the pixels from the edges of the image to 0
    img[0,:] = 0
    img[-1,:] = 0
    img[:,0] = 0
    img[:,-1] = 0
    
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    output = cv2.connectedComponentsWithStats(thresh, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output
    
    cell_areas = []

    if not os.path.exists(path):
        os.mkdir(path)
    

    for i in range(0, numLabels):
        # if this is the first component then we examine the
        # *background* (typically we would just ignore this
        # component in our loop)
        if i == 0:
            text = "examining component {}/{} (background)".format(
                i + 1, numLabels)
        # otherwise, we are examining an actual connected component
        else:
            text = "examining component {}/{}".format( i + 1, numLabels)
        # print a status message update for the current connected
        # component
        print("[INFO] {}".format(text))
        # extract the connected component statistics and centroid
        area = stats[i, cv2.CC_STAT_AREA]
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        margin = 20

        componentMask = (labels == i).astype("uint8") * 255
        #crop the image
        #componentMask = componentMask[y1:y2,x1:x2]
        #check how centered is the centroid
        if i!=0 and area > 50:        
            #save the image
            #check if the margin is out of bounds
            if x-margin < 0:
                x = margin
            if y-margin < 0:
                y = margin
            if x+w+margin > segmented.shape[1]:
                x = segmented.shape[1] - w - margin
            if y+h+margin > segmented.shape[0]:
                y = segmented.shape[0] - h - margin
            
            #Esto solo para el bounding box en nuestro caso al ser imagees tan zoomeadas no es necesario
            #componentMask = componentMask[y-margin:y+h+margin,x-margin:x+w+margin]
            
            imag_save.imsave(path+str(j)+"cell"+str(i)+".png", componentMask,cmap="gray")
            

    return np.array(cell_areas).astype(np.float32)

In [ ]:
#use separar regiones on all the images in the folder
separated_path = "path_of_separated_masks"


#remove all directories from save_path
for i in os.listdir(separated_path):
    #remove all files from the directory
    for j in os.listdir(separated_path+"/"+i):
        os.remove(separated_path+"/"+i+"/"+j)
    os.rmdir(separated_path+"/"+i)

for i in os.listdir(save_path):
    if not os.path.isfile(separated_path+"/"+i):
        os.mkdir(separated_path+"/"+i)
    for j in os.listdir(save_path+"/"+i):
        #read the image via numpy
        
        img = cv2.imread(save_path+"/"+i+"/"+j,0)
        aux = separar_regiones(img,path=separated_path+"/"+i+"/",j=j)



PREPARACIÓN DE LOS DATOS

In [ ]:
sep_path = "path_of_separated_masks"
dataset_path = "path_of_separated_masks"
dirs = ["bipiramides","hexagonos","circulos","cuadrados","rectangulos"]



In [ ]:
sep_path = "path_of_separated_masks"

dirs = ["bipiramides","hexagonos","circulos","cuadrados","rectangulos"]
#remove all directories from save_path
for i in os.listdir(sep_path):
    #remove all files from the directory
    for j in os.listdir(sep_path+"/"+i):
        os.remove(sep_path+"/"+i+"/"+j)
    os.rmdir(sep_path+"/"+i)

for i in dirs:
    os.mkdir(sep_path+"/"+i)

In [ ]:
dataset_path = "path_of_separated_masks"

#remove all repeated images on the same folder
for i in os.listdir(dataset_path):
    for j in os.listdir(dataset_path+"/"+i):
        for k in os.listdir(dataset_path+"/"+i):
            if j!=k:
                if os.path.isfile(dataset_path+"/"+i+"/"+j) and os.path.isfile(dataset_path+"/"+i+"/"+k):
                    if np.array_equal(cv2.imread(dataset_path+"/"+i+"/"+j,0),cv2.imread(dataset_path+"/"+i+"/"+k,0)):
                        os.remove(dataset_path+"/"+i+"/"+k)


In [ ]:
#DO DATA AUGMENTATION
dataset_path = "path_of_separated_masks"

samples = 140

#iterate over all the folders
for i in os.listdir(dataset_path):
    for j in os.listdir(dataset_path+"/"+i):
        #remove all the files which contains flip in the name
        if "flip" in j:
            os.remove(dataset_path+"/"+i+"/"+j)


#iterate over all the folders
for i in os.listdir(dataset_path):
    #if teh folder has less tan 100 images do data augmentation
    if len(os.listdir(dataset_path+"/"+i)) < 140:
        k=0

        while len(os.listdir(dataset_path+"/"+i)) < 140:
            
        #iterate over all the images in the folder
            for j in os.listdir(dataset_path+"/"+i):

                #read the image
                img = cv2.imread(dataset_path+"/"+i+"/"+j,0)
                
                angle = np.random.randint(0,360)
                rows,cols = img.shape
                M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
                
                img = cv2.warpAffine(img,M,(cols,rows))
                #reflect the image
                rep = None
                ref = np.random.randint(0,3)
                if ref == 1:
                    img = cv2.flip(img,0)
                elif ref == 2:
                    img = cv2.flip(img,1)

                #save the image
                imag_save.imsave(dataset_path+"/"+i+"/"+str(k)+"flip.png", img,cmap="gray")
                k+=1
                if len(os.listdir(dataset_path+"/"+i)) >= 140:
                    break

In [ ]:
#LOAD THE DATASET FROM THE DIRECTORY
#load the images
dataset_path = "path_of_separated_masks"
#create the datagen with dtype uint8
data_x = []
data_y = []
for i in os.listdir(dataset_path):
    #check if is a folder skip if not
    if not os.path.isdir(dataset_path+"/"+i):
        continue
    #iterate over all images in the subfolder
    for j in os.listdir(dataset_path+"/"+i):
        L = [0]*len(dirs)
        #read the image witout resizing
        img = cv2.imread(dataset_path+"/"+i+"/"+j,0)
        
        img = cv2.resize(img,(64,64))
        #convert the image to a numpy array
        img = np.array(img)
        #append the image to the dataset
        data_x.append(img)
        #append the label to the dataset
        L[dirs.index(i)] = 1
        data_y.append(L)


In [ ]:
data_x = np.array(data_x)
data_y = np.array(data_y)

In [ ]:
def prepare_data(size,dataset_path):
    data_x = []
    data_y = []
    for i in os.listdir(dataset_path):
        #check if is a folder skip if not
        if not os.path.isdir(dataset_path+"/"+i):
            continue
        #iterate over all images in the subfolder
        for j in os.listdir(dataset_path+"/"+i):
            L = [0]*len(dirs)
            #read the image witout resizing
            img = cv2.imread(dataset_path+"/"+i+"/"+j,0)
            
            img = cv2.resize(img,size)
            #convert the image to a numpy array
            img = np.array(img)
            #append the image to the dataset
            data_x.append(img)
            #append the label to the dataset
            L[dirs.index(i)] = 1
            data_y.append(L)
    return np.array(data_x),np.array(data_y)

In [ ]:
def get_model_denso(input_shape):
    
    modelo_denso = tf.keras.Sequential()

    modelo_denso.add(tf.keras.layers.Flatten(input_shape=input_shape))

    modelo_denso.add(tf.keras.layers.Dense(512,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(256,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(256,activation='relu'))
    
    modelo_denso.add(tf.keras.layers.Dense(128,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(64,activation='relu'))
    
    modelo_denso.add(tf.keras.layers.Dense(32,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(5,activation='softmax'))


    modelo_denso.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return modelo_denso

In [ ]:
def get_model_denso_prueba_pardo(input_shape):
    
    modelo_denso = tf.keras.Sequential()

    modelo_denso.add(tf.keras.layers.Flatten(input_shape=input_shape))

    modelo_denso.add(tf.keras.layers.Dense(2048,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(512,activation='relu'))

    #modelo_denso.add(tf.keras.layers.Dense(512,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(256,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(256,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(128,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(128,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(64,activation='relu'))

    modelo_denso.add(tf.keras.layers.Dense(5,activation='softmax'))

    modelo_denso.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return modelo_denso

In [ ]:
dataset_path = "path_of_separated_masks"
testing_path = "path_of_separated_TESTING_masks"

#remove all repeated images on the same folder para separar datos de entrenamiento y test
for i in os.listdir(dataset_path):
    for j in os.listdir(dataset_path+"/"+i):
        for k in os.listdir(testing_path+"/"+i):
            if j!=k:
                if os.path.isfile(dataset_path+"/"+i+"/"+j) and os.path.isfile(testing_path+"/"+i+"/"+k):
                    if np.array_equal(cv2.imread(dataset_path+"/"+i+"/"+j,0),cv2.imread(testing_path+"/"+i+"/"+k,0)):
                        os.remove(dataset_path+"/"+i+"/"+k)

#validation data
validation_path = "path_of_separated_validation_masks"

for i in os.listdir(dataset_path):
    for j in os.listdir(dataset_path+"/"+i):
        for k in os.listdir(validation_path+"/"+i):
            if j!=k:
                if os.path.isfile(dataset_path+"/"+i+"/"+j) and os.path.isfile(validation_path+"/"+i+"/"+k):
                    if np.array_equal(cv2.imread(dataset_path+"/"+i+"/"+j,0),cv2.imread(validation_path+"/"+i+"/"+k,0)):
                        os.remove(dataset_path+"/"+i+"/"+k)

In [ ]:
size = (100,100)

modelo = get_model_denso_prueba_pardo(size)

data_x,data_y = prepare_data(size,"path_of_separated_masks")

test_x,test_y = prepare_data(size,"path_of_separated_test_masks")

val_x,val_y = prepare_data(size,"path_of_separated_validation_masks")

#callback for saving the best model
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./Recursos/Clasificacion/Modelos_Clasificacion/modelo_denso_prueba_variacion_pardo.h5',monitor='val_accuracy',save_best_only=True)]

history = modelo.fit(data_x,data_y,epochs=50,batch_size=100,shuffle=True,verbose=1,validation_data=(test_x,test_y),callbacks=callbacks)

In [ ]:
#plot the accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','val'],loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train','val'],loc='upper left')
plt.show()


In [ ]:
val_x,val_y = prepare_data(size,"path_of_separated_validation_masks")

In [ ]:
test_sizes = [(i,i) for i in range(50,121,10)]

In [ ]:
medidas = []


for size in test_sizes:
    print("Tamaño de la imagen: ",size)
    accuracy = None
    loss = None

    for i in range(3):
        modelo = get_model_denso_prueba_pardo(size)

        train_x,train_y = prepare_data(size,"path_of_separated_masks")

        val_x,val_y = prepare_data(size,"path_of_separated_validation_masks")

        modelo.fit(train_x,train_y,epochs=50,shuffle=True,verbose=1)

        aux_loss,aux_accuracy = modelo.evaluate(val_x,val_y,verbose=0)
        if accuracy == None or aux_accuracy > accuracy:
            accuracy = aux_accuracy
            loss = aux_loss
            modelo.save("./Recursos/Clasificacion/Modelos_Clasificacion/modelo_denso_prueba_pardo"+ str(size[0])+".h5")
            print("Loss: ",loss)
            print("Accuracy: ",accuracy)

    medidas.append((loss,accuracy))



plt.plot([i[0] for i in test_sizes],[i[1] for i in medidas])
plt.xlabel("Tamaño de la imagen")
plt.ylabel("Accuracy")
plt.show()

plt.plot([i[0] for i in test_sizes],[i[0] for i in medidas])
plt.xlabel("Tamaño de la imagen")
plt.ylabel("Loss")
plt.show()

In [ ]:
import random

plt.plot([i[0] for i in test_sizes],[i[1] for i in medidas])
plt.xlabel("Tamaño de la imagen")
plt.ylabel("Accuracy")
plt.show()

plt.plot([i[0] for i in test_sizes],[i[0] for i in medidas])
plt.xlabel("Tamaño de la imagen")
plt.ylabel("Loss")
plt.show()